## 

+ 今年3/23が底値
+ そこから、2020/06/12までで、戻してる銘柄の特徴を得る


In [ ]:
MORNINGSTAR_INDUSTRY_GROUP_CODES_after_Nov2019 =  {10110: 'Agricultural Inputs',
 10120: 'Building Materials',
 10130: 'Chemicals,Specialty Chemicals',
 10140: 'Lumber & Wood Production,Paper & Paper Products',
 10150: 'Aluminum,Copper,Other Industrial Metals & Mining,Gold,Silver,Other Precious Metals & Mining',
 10160: 'Coking Coal,Steel',
 10200: 'Auto & Truck Dealerships,Auto Manufacturers,Auto Parts,Recreational Vehicles',
 10220: 'Furnishings, Fixtures & Appliances',
 10230: 'Residential Construction',
 10240: 'Textile Manufacturing,Apparel Manufacturing,Footwear & Accessories',
 10250: 'Packaging & Containers',
 10260: 'Personal Services',
 10270: 'Restaurants',
 10280: 'Apparel Retail,Department Stores,Home Improvement Retail,Luxury Goods,Internet Retail,Specialty Retail',
 10290: 'Gambling,Leisure,Lodging,Resorts & Casinos,Travel Services',
 10310: 'Asset Management',
 10320: 'Banks—Diversified,Banks—Regional,Mortgage Finance',
 10330: 'Capital Markets,Financial Data & Stock Exchanges',
 10340: 'Insurance—Life,Insurance—Property & Casualty,Insurance—Reinsurance,Insurance—Specialty,Insurance Brokers,Insurance—Diversified',
 10350: 'Shell Companies,Financial Conglomerates',
 10360: 'Credit Services',
 10410: 'Real Estate—Development,Real Estate Services,Real Estate—Diversified',
 10420: 'REIT—Healthcare Facilities,REIT—Hotel & Motel,REIT—Industrial,REIT—Office,REIT—Residential,REIT—Retail,REIT—Mortgage,REIT—Specialty,REIT—Diversified',
 20510: 'Beverages—Brewers,Beverages—Wineries & Distilleries',
 20520: 'Beverages—Non-Alcoholic',
 20525: 'Confectioners,Farm Products,Household & Personal Products,Packaged Foods',
 20540: 'Education & Training Services',
 20550: 'Discount Stores,Food Distribution,Grocery Stores',
 20560: 'Tobacco',
 20610: 'Biotechnology',
 20620: 'Drug Manufacturers—General,Drug Manufacturers—Specialty & Generic',
 20630: 'Healthcare Plans',
 20645: 'Medical Care Facilities,Pharmaceutical Retailers,Health Information Services',
 20650: 'Medical Devices,Medical Instruments & Supplies',
 20660: 'Diagnostics & Research',
 20670: 'Medical Distribution',
 20710: 'Utilities—Independent Power Producers,Utilities—Renewable',
 20720: 'Utilities—Regulated Water,Utilities—Regulated Electric,Utilities—Regulated Gas,Utilities—Diversified',
 30810: 'Telecom Services',
 30820: 'Advertising Agencies,Publishing,Broadcasting,Entertainment',
 30830: 'Internet Content & Information,Electronic Gaming & Multimedia',
 30910: 'Oil & Gas Drilling,Oil & Gas E&P,Oil & Gas Integrated,Oil & Gas Midstream,Oil & Gas Refining & Marketing,Oil & Gas Equipment & Services',
 30920: 'Thermal Coal,Uranium',
 31010: 'Aerospace & Defense',
 31020: 'Specialty Business Services,Consulting Services,Rental & Leasing Services,Security & Protection Services,Staffing & Employment Services',
 31030: 'Conglomerates',
 31040: 'Engineering & Construction,Infrastructure Operations,Building Products & Equipment',
 31050: 'Farm & Heavy Construction Machinery',
 31060: 'Industrial Distribution',
 31070: 'Business Equipment & Supplies,Specialty Industrial Machinery,Metal Fabrication,Pollution & Treatment Controls,Tools & Accessories,Electrical Equipment & Parts',
 31080: 'Airports & Air Services,Airlines,Railroads,Marine Shipping,Trucking,Integrated Freight & Logistics',
 31090: 'Waste Management',
 31110: 'Information Technology Services,Software—Application,Software—Infrastructure',
 31120: 'Communication Equipment,Computer Hardware,Consumer Electronics,Electronic Components,Electronics & Computer Distribution,Scientific & Technical Instruments',
 31130: 'Semiconductor Equipment & Materials,Semiconductors,Solar'}

## 現在QTradableStocksUSに入っている銘柄を確認

+ QTradableStocksUS
+ financial_health_grade
+ morningstar_industry_group_code

でグループ分け


In [ ]:
import numpy as np
import pandas as pd
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.morningstar import Fundamentals
from quantopian.pipeline.domain import US_EQUITIES
from quantopian.research import run_pipeline
from quantopian.pipeline.filters import Q500US, StaticSids, StaticAssets
from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline.data import USEquityPricing
from quantopian.pipeline.factors import Returns

from quantopian.pipeline.classifiers.fundamentals import Sector


def make_pipeline():
    
    base_universe = QTradableStocksUS()
    close_price = USEquityPricing.close.latest

    morningstar_industry_group_code = Fundamentals.morningstar_industry_group_code.latest
    prev_change = Returns(inputs = [USEquityPricing.close], window_length=2)
    financial_health_grade = Fundamentals.financial_health_grade.latest
    
    
    
    pipe = Pipeline(
        columns={
            'morningstar_industry_group_code':morningstar_industry_group_code, 
            'close_price':close_price, 
            "prev_change": prev_change,
            'financial_health_grade':financial_health_grade,
        },
        
        screen=base_universe,
    )
    return pipe 

df = run_pipeline(make_pipeline(), '2020-01-01', '2020-06-11')
df["sector_name"] = df["morningstar_industry_group_code"].replace(MORNINGSTAR_INDUSTRY_GROUP_CODES_after_Nov2019)

df.head()

## 直近のグレードと morningstar_industry_group_code でグループ化

In [ ]:
dict_group = dict(list(df.loc["2020-06-11"].groupby(["morningstar_industry_group_code", "financial_health_grade"])))

In [ ]:
dict_group.keys()

### sector 毎YTDのパフォーマンスを確認

In [ ]:
df.groupby("morningstar_industry_group_code")[["prev_change"]].sum().sort_values(by="prev_change").head()

In [ ]:
df.groupby("morningstar_industry_group_code")[["prev_change"]].sum().sort_values(by="prev_change").tail()

### sector別成績で上位と下位のA-F銘柄を比べる

In [ ]:
from quantopian.research import prices, symbols
start='2020-01-01'
end='2020-06-11'

def get_symbols(group_num,  financial_grade,  dict_group=dict_group, spy=True):
    x = []
    
    if spy:
        x =  [symbols("SPY")]

    
    return dict_group[(group_num,financial_grade )]["prev_change"].unstack().columns.tolist() + x

def get_prices(symbols, start, end):
    return prices(assets=symbols,start=start, end=end,)

def plot(prices, top=False, bottom=False, spy=True):
    x = []
    if spy:
        x =  [symbols("SPY")]
    
    cum = (prices / prices.shift(1)).apply(np.log).cumsum()

    sym = None
    if bottom:
        sym = cum.iloc[-1].sort_values().head().index.tolist() + x
    elif top:
        sym = cum.iloc[-1].sort_values().tail().index.tolist() + x
    
    if sym:
        print([s.symbol for s in sym])
        cum[sym].plot()
    else:
        cum.plot()
    
        

symbol_enegry_A = get_symbols(30910,"A")
symbol_enegry_D = get_symbols(30910,"D")
df_enegry_A = get_prices(symbol_enegry_A, start, end)
df_enegry_D = get_prices(symbol_enegry_D, start, end)

symbol_restaulant_A = get_symbols(10270,"A")
symbol_restaulant_D = get_symbols(10270,"D")

df_restaurant_A = get_prices(symbol_restaulant_A, start, end)
df_restaurant_D = get_prices(symbol_restaulant_D, start, end)


In [ ]:
plot(df_restaurant_A, top=True, spy=False)
plot(df_restaurant_D, bottom=True, spy=False)


In [ ]:

long = ['YUMC', 'CMG', 'WING']
short = ['PLAY', 'BLMN', 'EAT']

df_long = get_prices([symbols(s) for s in long], start, end).apply(np.log)
df_short = get_prices([symbols(s) for s in short], start, end).apply(np.log)

df_long



In [ ]:
pd.concat([df_long/df_long.shift(), df_short.shift(1)/ df_short], axis=1)